In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from random import seed
from random import randint
import random
import string
import re
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns
import transformers

import nltk


from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

plt.style.use('seaborn')

In [ ]:
import nltk
nltk.download('punkt')
nltk.download("stopwords")
nltk.download("words")
from nltk.corpus import stopwords
from nltk.corpus import words

In [ ]:
from tensorflow.keras.layers import Dense, Input, Reshape, Conv1D, Conv2D, BatchNormalization, MaxPooling1D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, Sequential
# from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [ ]:
seed(1)
stop_words = stopwords.words('english')
words=words.words()

In [ ]:
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

In [ ]:
PATH_CSV_TRAIN = '../input/openofficepreprocessed6/openoffice_preprocessed.csv'
# PATH_CSV_TEST = '../input/nlpgettingstarted/test.csv'
# PATH_CSV_SUBMISSION = '../input/nlpgettingstarted/sample_submission.csv'

dataset = pd.read_csv(PATH_CSV_TRAIN)
# dataf_test = pd.read_csv(PATH_CSV_TEST)

In [ ]:
dataset.dropna(subset = ["description2"], inplace=True)
dataset.dropna(subset = ["short_desc2"], inplace=True)

In [ ]:
def reset_column_names():
  dataset.drop('description1', axis=1, inplace=True)
  dataset.drop('short_desc1', axis=1, inplace=True)
  dataset.drop('description2', axis=1, inplace=True)
  dataset.drop('short_desc2', axis=1, inplace=True)

  dataset.rename(columns={'description1_clean':'description1','short_desc1_clean':'short_desc1','description2_clean':'description2','short_desc2_clean':'short_desc2'}, inplace=True)

In [ ]:
def clean_text(text):
    clean=text
    
    reg = re.compile(r" +")
    clean = clean.apply(lambda r: re.sub(reg, string=r, repl=' '))

    #Lowercase
    clean = clean.apply(lambda r: r.lower())
    return clean

In [ ]:
dataset['description1_clean'] = clean_text(dataset['description1'])
dataset['short_desc1_clean'] = clean_text(dataset['short_desc1'])
dataset['description2_clean'] = clean_text(dataset['description2'])
dataset['short_desc2_clean'] = clean_text(dataset['short_desc2'])
dataset.head()

In [ ]:
reset_column_names()

In [ ]:
for index, row in dataset.iterrows():
    desc = row['description1'] + row['description2']
    dataset.loc[index, 'description'] = desc

In [ ]:
from transformers import TFXLNetModel, XLNetTokenizer, XLNetConfig

In [ ]:
xlnet_model = 'xlnet-base-cased'
xlnet_tokenizer = XLNetTokenizer.from_pretrained(xlnet_model)

In [ ]:
def create_xlnet(mname):
    """ Creates the model. It is composed of the XLNet main block and then
    a classification head its added
    """
    # Define token ids as inputs
    word_inputs = tf.keras.Input(shape=(120,), name='word_inputs', dtype='int32')

    # Call XLNet model
    xlnet = TFXLNetModel.from_pretrained(mname)
    xlnet_encodings = xlnet(word_inputs)[0]

    # CLASSIFICATION HEAD 
    # Collect last step from last hidden state (CLS)
    doc_encoding = tf.squeeze(xlnet_encodings[:, -1:, :], axis=1)
    print(doc_encoding)
    # Apply dropout for regularization
    doc_encoding = tf.keras.layers.Dropout(.1)(doc_encoding)
    # Final output 
    outputs = tf.keras.layers.Dense(1, activation='sigmoid', name='outputs')(doc_encoding)

    # Compile model
    model = tf.keras.Model(inputs=[word_inputs], outputs=[outputs])
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=2e-5), loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

    return model

In [ ]:
def get_inputs(text, tokenizer, max_len=120):
    """ Gets tensors from text using the tokenizer provided"""
#     text = [w.lower() for w in text]
#     text = [w for w in text if w.lower() not in stop_words]
#     text = [w for w in text if w in words or w.isalpha()]
    inps = [tokenizer.encode_plus(t, max_length=max_len, pad_to_max_length=True, add_special_tokens=True) for t in text]
    inp_tok = np.array([a['input_ids'] for a in inps])
    ids = np.array([a['attention_mask'] for a in inps])
    segments = np.array([a['token_type_ids'] for a in inps])
    return inp_tok, ids, segments

In [ ]:
X = dataset['description']
Y = np.array(dataset.duplicate.values, dtype='int')

kfold = StratifiedKFold()
cvscores = []
    
for train, test in kfold.split(X, Y):

    # Prepare data
    inp_tok, ids, segments = get_inputs(X.iloc[train], xlnet_tokenizer)
    y_train = np.array(Y[train], dtype='int')
    
    xlnet = create_xlnet(xlnet_model)
    xlnet.summary()
    # Fit the model  
    hist = xlnet.fit(x=inp_tok, y=y_train, epochs=5, batch_size=10, validation_split=.15)
    
    test_inp_tok, test_ids, test_segments = get_inputs(X.iloc[test], xlnet_tokenizer)
    y_test = np.array(Y[test], dtype='int')

    # Evaluate the model
    scores = xlnet.evaluate(test_inp_tok,y_test, verbose=0)
    f1score = (2 * scores[2]*100 * scores[3]*100)/(scores[2]*100 + scores[3]*100)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    print("%s: %.2f%%" % (model.metrics_names[2], scores[2]*100))
    print("%s: %.2f%%" % (model.metrics_names[3], scores[3]*100))
    print("f1 score: ", (f1score))
    cvscores.append(scores[1] * 100)
    precision_scores.append(scores[2] * 100)
    recall_scores.append(scores[3] * 100)
    f1_scores.append(f1score)

print("accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
print("precision: %.2f%% (+/- %.2f%%)" % (np.mean(precision_scores), np.std(precision_scores)))
print("recall: %.2f%% (+/- %.2f%%)" % (np.mean(recall_scores), np.std(recall_scores)))
print("f1 score: %.2f%% (+/- %.2f%%)" % (np.mean(f1_scores), np.std(f1_scores)))